In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from clase import Libro
from sklearn.preprocessing import RobustScaler

In [ ]:

MEP = pd.read_csv('market_data_AL30/AL30_1205_CI_MEP.csv')

MEP.drop(columns=['settlement', 'key', 'symbol'], inplace=True)

In [3]:
features_names = ['time_since_last_trade',
                             'time_since_last_update',
                             'mid_price',
                             'spread',
                             'volumen_bid',
                             'volumen_off',
                             'imbalance',
                             'vwap_bid',
                             'vwap_offer',
                             'relative_spread',
                             'log_spread',
                             'price_imbalance',
                             'depth_ratio',
                             'direccion_bid',
                             'direccion_offer']

In [4]:
Libro_MEP = Libro(MEP)
filas = []
contador = 1
for _ in range(200):
    Libro_MEP.actualizar()
while Libro_MEP.indice < 1601500:
    Libro_MEP.actualizar()
    features = Libro_MEP.obtener_features()
    filas.append(features)

In [5]:
df = pd.DataFrame(np.vstack(filas), columns=features_names)

In [19]:
#Ahora genero la columna target. Esta va a ser el log return del mid price con una ventada de 40 operaciones. 
h = 40

mid_prices = df['mid_price'].astype(float).values
log_returns = [np.log(mid_prices[i + h] / mid_prices[i]) for i in range(len(mid_prices) - h)]
log_returns += [None] * h 


df['log_return'] = log_returns

In [ ]:
pos = 262200
h1=40
mid_prices = df['mid_price'].astype(float).values

plt.figure(figsize=(12, 5))

# Plot the first segment
plt.subplot(1, 2, 1)
plt.plot(mid_prices[pos-100:pos])
plt.title("Segment 1")

# Plot the second segment
plt.subplot(1, 2, 2)
plt.plot(mid_prices[pos:pos+h1])
plt.title("Segment 2")

plt.tight_layout()

In [20]:
df.to_csv('Data_MEP', index=False)

Ahora quiero normalizar los datos con escalas muy grandes. Luego los datos de normalización los incorporo adentro de la red

In [6]:
#convierto los nanosegundos a segundos
df['time_since_last_trade'] = df['time_since_last_trade'] / 1e7
df['time_since_last_update'] = df['time_since_last_update'] / 1e7


depth_ratio = df['depth_ratio'].astype(float).values.reshape(-1,1)
scaler_depth_ratio = RobustScaler()
scaler_depth_ratio.fit(depth_ratio)
print(f"Datos Para normalizar el depth ratio al input de la red: x - {scaler_depth_ratio.center_} / {scaler_depth_ratio.scale_}")

price_imbalance = df['price_imbalance'].astype(float).values.reshape(-1,1)
scaler_price_imbalance = RobustScaler()
scaler_price_imbalance.fit(price_imbalance)
print(f"Datos Para normalizar el price imbalance al input de la red: x - {scaler_price_imbalance.center_} / {scaler_price_imbalance.scale_}")

vol_bid = df['volumen_bid'].astype(int).values
vol_off = df['volumen_off'].astype(int).values
volumenes = np.concatenate((vol_off, vol_bid), axis=0)
volumenes = volumenes.reshape(-1,1)
scaler_vol = RobustScaler()
scaler_vol.fit(volumenes)
print(f"Datos Para normalizar el volumen al input de la red: x - {scaler_vol.center_} / {scaler_vol.scale_}")

Datos Para normalizar el depth ratio al input de la red: x - [-120097.] / [494227.]
Datos Para normalizar el price imbalance al input de la red: x - [-1.6963328e-06] / [8.28752812e-06]
Datos Para normalizar el volumen al input de la red: x - [573690.] / [515196.]
